# The Commutator Zoo: Non-Commutative Geometry Across Physics & AI

## The Central Dogma: Order Matters

In everyday arithmetic, we're used to operations that commute:
$$3 \times 5 = 5 \times 3$$

But in the real world—rotations, quantum mechanics, control theory, robotics, and even deep learning—**order matters**. The mathematical measure of this "failure to commute" is the **commutator**:

$$[A, B] = AB - BA$$

If $[A, B] \neq 0$, then performing operation $A$ followed by $B$ gives a different result than $B$ followed by $A$.

This notebook explores five distinct domains where non-commutativity creates surprising and powerful phenomena:

1. **Cars (SE(2))**: Parking sideways using Lie brackets
2. **Cats (SO(3))**: Landing on feet using gauge theory
3. **Claws (SE(3))**: Robot arms navigating 3D space
4. **Kets (SU(2))**: Quantum gates on the Bloch sphere
5. **Continuous (Diff)**: Neural ODEs and non-commutative flows

---

In [ ]:
# Import all necessary libraries
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from scipy.integrate import solve_ivp
import warnings
warnings.filterwarnings('ignore')

# Set matplotlib style
plt.style.use('seaborn-v0_8-darkgrid')
%matplotlib inline

## Summary Table: The Five Examples

| Module | Domain | Group | Manifold | Commutator Effect |
|:-------|:-------|:------|:---------|:------------------|
| **Cars** | Control Theory | $SE(2)$ | 2D Plane | **Sideways Slide** (wiggling into tight spaces) |
| **Cats** | Biomechanics | $SO(3)$ | Shape Space | **Net Rotation** (landing on feet with zero angular momentum) |
| **Claws** | Robotics | $SE(3)$ | 3D Rigid Body | **End-Effector Displacement** (path-dependent positioning) |
| **Kets** | Quantum Physics | $SU(2)$ | Hilbert Space | **Geometric Phase** (non-commuting quantum gates) |
| **Continuous** | Deep Learning | Diff | Neural Manifold | **Different Inference** (layer order matters) |

---

# 1. Cars: The Lie Bracket ($SE(2)$)

## The Problem

A car has 3 degrees of freedom: position $(x, y)$ and orientation $\theta$. However, it only has 2 controls:
- **Drive**: Move forward/backward
- **Steer**: Change orientation

A car **cannot move directly sideways**. Yet somehow, we can parallel park!

## The Mathematics

The car's configuration space is $SE(2)$, the **Special Euclidean Group in 2D**. The operations "Drive" and "Steer" do not commute:

$$[\text{Drive}, \text{Steer}] = \text{Sideways Slide}$$

By performing the sequence $\text{Steer} \to \text{Drive} \to \text{Un-Steer} \to \text{Un-Drive}$, we generate net motion in the "forbidden" direction.

This is the **Brockett Integrator** from control theory—a fundamental example of non-holonomic constraints.

## The Code

In [ ]:
def apply_commutator_parking(num_cycles=1, angle=np.pi/4):
    """
    Simulate parallel parking using the commutator [Steer, Drive].
    
    Parameters:
    -----------
    num_cycles : int
        Number of commutator cycles to perform
    angle : float
        Steering angle (radians)
    """
    # Initial State [x, y, theta]
    state = np.array([0.0, 0.0, 0.0])
    path = [state.copy()]
    dt = 0.01
    
    def update_state(current_state, v, w, duration):
        """Update car state using velocity v and angular velocity w."""
        steps = int(duration / dt)
        temp_state = current_state.copy()
        for _ in range(steps):
            theta = temp_state[2]
            # Kinematic equations for a car
            temp_state[0] += v * np.cos(theta) * dt  # x
            temp_state[1] += v * np.sin(theta) * dt  # y
            temp_state[2] += w * dt                   # theta
            path.append(temp_state.copy())
        return temp_state
    
    # Perform multiple commutator cycles
    for cycle in range(num_cycles):
        # COMMUTATOR SEQUENCE: [A, B] = A B A^{-1} B^{-1}
        state = update_state(state, v=0, w=angle, duration=1.0)   # A: Steer
        state = update_state(state, v=1, w=0, duration=1.0)       # B: Drive
        state = update_state(state, v=0, w=-angle, duration=1.0)  # A^{-1}: Un-Steer
        state = update_state(state, v=-1, w=0, duration=1.0)      # B^{-1}: Un-Drive
    
    path = np.array(path)
    
    # Visualization
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))
    
    # Left plot: Car trajectory
    ax1.plot(path[:, 0], path[:, 1], 'b-', linewidth=2, alpha=0.6, label='Path')
    ax1.scatter(0, 0, color='green', s=200, marker='o', label='Start', zorder=5)
    ax1.scatter(path[-1, 0], path[-1, 1], color='red', s=200, marker='s', label='End', zorder=5)
    
    # Draw the commutator displacement
    ax1.arrow(0, 0, path[-1, 0], path[-1, 1], 
              width=0.01, head_width=0.05, head_length=0.05,
              color='orange', alpha=0.7, label='Net Displacement', zorder=4)
    
    ax1.set_title(f'Cars (SE(2)): The Lie Bracket [Drive, Steer]\nCycles: {num_cycles}', fontsize=14, fontweight='bold')
    ax1.set_xlabel('x position', fontsize=12)
    ax1.set_ylabel('y position', fontsize=12)
    ax1.grid(True, alpha=0.3)
    ax1.legend(fontsize=11)
    ax1.axis('equal')
    
    # Right plot: Component trajectories
    time = np.arange(len(path)) * dt
    ax2.plot(time, path[:, 0], 'r-', label='x(t)', linewidth=2)
    ax2.plot(time, path[:, 1], 'g-', label='y(t)', linewidth=2)
    ax2.plot(time, path[:, 2], 'b-', label='θ(t)', linewidth=2)
    ax2.set_title('State Variables Over Time', fontsize=14, fontweight='bold')
    ax2.set_xlabel('Time (s)', fontsize=12)
    ax2.set_ylabel('State Value', fontsize=12)
    ax2.legend(fontsize=11)
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print(f"\nFinal displacement after {num_cycles} cycle(s):")
    print(f"  Δx = {path[-1, 0]:.6f}")
    print(f"  Δy = {path[-1, 1]:.6f}  ← Sideways motion!")
    print(f"  Δθ = {path[-1, 2]:.6f}")

# Run the simulation
apply_commutator_parking(num_cycles=3, angle=np.pi/4)

### Key Insight: Cars

Notice that even though we start and end at the same orientation ($\theta$), we've achieved **net sideways displacement** in the $y$ direction. This is impossible with any single control input, but the commutator $[\text{Steer}, \text{Drive}]$ generates motion in this "forbidden" direction.

This is exactly how parallel parking works: by alternating steering and driving, we can slide sideways into a tight space.

---

# 2. Cats: Gauge Theory ($SO(3)$)

## The Problem

A falling cat has **zero angular momentum** ($L = 0$). By conservation of angular momentum, it should not be able to rotate. Yet cats reliably land on their feet!

## The Mathematics

The cat lives in a **Principal Bundle**:
- **Base Space**: The cat's internal shape (bend angle, twist angle)
- **Fiber**: The cat's orientation in space ($SO(3)$)

When the cat changes its shape in a closed loop (Bend → Twist → Un-Bend → Un-Twist), it generates a **holonomy**—a net rotation in the fiber, even though the loop closes in the base space.

This is mathematically identical to:
- An electron moving through a magnetic field (Berry phase)
- Parallel transport on a curved manifold
- The Aharonov-Bohm effect in quantum mechanics

## The Code: Brockett Integrator (Shape Space)

In [ ]:
def run_brockett(duration=4*np.pi, resolution=1000):
    """
    Simulate the Brockett integrator: [x, y] generates motion in z.
    This models how changing shape (x, y) creates rotation (z).
    
    Parameters:
    -----------
    duration : float
        Total time for the simulation
    resolution : int
        Number of time steps
    """
    t = np.linspace(0, duration, resolution)
    dt = t[1] - t[0]
    
    # Control inputs (shape changes)
    u1 = np.cos(t)  # "Bend" control
    u2 = np.sin(t)  # "Twist" control
    
    # State variables
    x = np.zeros_like(t)  # Bend state
    y = np.zeros_like(t)  # Twist state
    z = np.zeros_like(t)  # Rotation (orientation)
    
    # Integrate the Brockett system
    for i in range(1, len(t)):
        x[i] = x[i-1] + u1[i-1] * dt
        y[i] = y[i-1] + u2[i-1] * dt
        # The magic: z changes proportional to the area swept in (x, y) space
        # This is the commutator term: [x, y] contributes to z
        z[i] = z[i-1] + (x[i-1] * u2[i-1] - y[i-1] * u1[i-1]) * dt
    
    # Create visualizations
    fig = plt.figure(figsize=(18, 5))
    
    # 3D trajectory in shape-orientation space
    ax1 = fig.add_subplot(131, projection='3d')
    ax1.plot(x, y, z, linewidth=2, color='darkblue', alpha=0.7)
    ax1.scatter(x[0], y[0], z[0], color='green', s=100, label='Start')
    ax1.scatter(x[-1], y[-1], z[-1], color='red', s=100, label='End')
    ax1.set_xlabel('Bend (x)', fontsize=11)
    ax1.set_ylabel('Twist (y)', fontsize=11)
    ax1.set_zlabel('Rotation (z)', fontsize=11)
    ax1.set_title('Cat Trajectory in Shape-Orientation Space', fontsize=12, fontweight='bold')
    ax1.legend()
    
    # Shape space (x, y) showing the closed loop
    ax2 = fig.add_subplot(132)
    ax2.plot(x, y, linewidth=2, color='purple')
    ax2.scatter(x[0], y[0], color='green', s=100, label='Start', zorder=5)
    ax2.scatter(x[-1], y[-1], color='red', s=100, label='End', zorder=5)
    ax2.set_xlabel('Bend (x)', fontsize=11)
    ax2.set_ylabel('Twist (y)', fontsize=11)
    ax2.set_title('Shape Space Loop\n(Closes!)', fontsize=12, fontweight='bold')
    ax2.grid(True, alpha=0.3)
    ax2.axis('equal')
    ax2.legend()
    
    # Rotation generated over time
    ax3 = fig.add_subplot(133)
    ax3.plot(t, z, linewidth=2, color='darkred')
    ax3.set_xlabel('Time', fontsize=11)
    ax3.set_ylabel('Rotation (z)', fontsize=11)
    ax3.set_title('Net Rotation Generated\n(Holonomy)', fontsize=12, fontweight='bold')
    ax3.grid(True, alpha=0.3)
    ax3.axhline(y=0, color='k', linestyle='--', alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print(f"\nBrockett Integrator Results:")
    print(f"  Shape loop closure: Δx = {x[-1]:.6f}, Δy = {y[-1]:.6f}")
    print(f"  Net rotation generated: Δz = {z[-1]:.6f} radians ({np.degrees(z[-1]):.2f}°)")
    print(f"\n  → The shape loop closes, but orientation changes!")
    print(f"  → This is the 'Holonomy' - rotation from shape deformation.")

# Run the simulation
run_brockett(duration=4*np.pi, resolution=1000)

### Key Insight: Cats

The shape space trajectory **closes** (the cat returns to the same internal configuration), but the orientation (rotation) **does not return to zero**. The cat has rotated by an angle proportional to the **area enclosed** by the shape trajectory.

This is gauge theory in action:
- The shape space is the "base space"
- The orientation is the "fiber" 
- Moving in a loop generates "holonomy" (net rotation)

The same mathematics describes:
- Berry phase in quantum mechanics
- Magnetic monopoles
- General relativity (parallel transport)

---

# 3. Claws: Rigid Body Motion ($SE(3)$)

## The Problem

Robot arms operate in 3D space, combining rotations and translations. The configuration space is $SE(3)$, the **Special Euclidean Group in 3D**.

## The Mathematics

We represent rigid body transformations using **4×4 homogeneous transformation matrices**:

$$T = \begin{bmatrix} R & t \\ 0 & 1 \end{bmatrix}$$

where $R$ is a 3×3 rotation matrix and $t$ is a 3×1 translation vector.

Matrix multiplication is **non-commutative**: $AB \neq BA$. This means:
- "Rotate then translate" $\neq$ "Translate then rotate"
- The order of joint movements matters for the final end-effector position

This is why robotics requires careful **forward kinematics** calculations.

## The Code

In [ ]:
def get_se3(theta, axis, translation):
    """
    Create a 4x4 homogeneous transformation matrix in SE(3).
    
    Parameters:
    -----------
    theta : float
        Rotation angle in radians
    axis : str
        Rotation axis ('x', 'y', or 'z')
    translation : array-like
        Translation vector [dx, dy, dz]
    
    Returns:
    --------
    T : ndarray (4, 4)
        Homogeneous transformation matrix
    """
    c, s = np.cos(theta), np.sin(theta)
    
    # Rotation matrices for each axis
    if axis == 'x':
        R = np.array([[1, 0, 0],
                      [0, c, -s],
                      [0, s, c]])
    elif axis == 'y':
        R = np.array([[c, 0, s],
                      [0, 1, 0],
                      [-s, 0, c]])
    elif axis == 'z':
        R = np.array([[c, -s, 0],
                      [s, c, 0],
                      [0, 0, 1]])
    
    # Build homogeneous transformation matrix
    T = np.eye(4)
    T[:3, :3] = R
    T[:3, 3] = translation
    return T

def simulate_se3_arm():
    """
    Demonstrate non-commutativity in SE(3) for robot kinematics.
    Compare two different orderings of rotations and translations.
    """
    # Starting position (homogeneous coordinates)
    origin = np.array([0, 0, 0, 1])
    
    # Define two operations
    angle = np.pi / 2  # 90 degrees
    
    # Operation A: Rotate around X-axis by 90° AND extend along Z
    Op_A = get_se3(angle, 'x', [0, 0, 1])
    
    # Operation B: Rotate around Y-axis by 90° AND extend along Z
    Op_B = get_se3(angle, 'y', [0, 0, 1])
    
    # Path 1: Apply A then B
    # Note: Matrix multiplication is right-to-left!
    pos_AB = Op_B @ Op_A @ origin
    
    # Path 2: Apply B then A
    pos_BA = Op_A @ Op_B @ origin
    
    # Also compute intermediate positions
    pos_A = Op_A @ origin
    pos_B = Op_B @ origin
    
    # Create visualization
    fig = plt.figure(figsize=(16, 7))
    
    # 3D plot showing the different paths
    ax1 = fig.add_subplot(121, projection='3d')
    
    # Draw coordinate frame
    ax1.quiver(0, 0, 0, 1, 0, 0, color='r', alpha=0.3, arrow_length_ratio=0.1, label='X-axis')
    ax1.quiver(0, 0, 0, 0, 1, 0, color='g', alpha=0.3, arrow_length_ratio=0.1, label='Y-axis')
    ax1.quiver(0, 0, 0, 0, 0, 1, color='b', alpha=0.3, arrow_length_ratio=0.1, label='Z-axis')
    
    # Path 1: Origin -> A -> AB
    ax1.plot([0, pos_A[0]], [0, pos_A[1]], [0, pos_A[2]], 
             'b--', linewidth=2, alpha=0.5, label='Step A')
    ax1.plot([pos_A[0], pos_AB[0]], [pos_A[1], pos_AB[1]], [pos_A[2], pos_AB[2]], 
             'b-', linewidth=3, label='Path A→B')
    
    # Path 2: Origin -> B -> BA
    ax1.plot([0, pos_B[0]], [0, pos_B[1]], [0, pos_B[2]], 
             'r--', linewidth=2, alpha=0.5, label='Step B')
    ax1.plot([pos_B[0], pos_BA[0]], [pos_B[1], pos_BA[1]], [pos_B[2], pos_BA[2]], 
             'r-', linewidth=3, label='Path B→A')
    
    # Highlight the commutator difference
    ax1.plot([pos_AB[0], pos_BA[0]], [pos_AB[1], pos_BA[1]], [pos_AB[2], pos_BA[2]], 
             'purple', linestyle='--', linewidth=3, marker='o', markersize=8,
             label='Commutator Difference')
    
    # Mark key points
    ax1.scatter(*origin[:3], color='green', s=200, marker='o', label='Origin', zorder=5)
    ax1.scatter(*pos_AB[:3], color='blue', s=200, marker='s', zorder=5)
    ax1.scatter(*pos_BA[:3], color='red', s=200, marker='^', zorder=5)
    
    ax1.set_xlabel('X', fontsize=11)
    ax1.set_ylabel('Y', fontsize=11)
    ax1.set_zlabel('Z', fontsize=11)
    ax1.set_title('SE(3): Robot Arm End-Effector Positions\n[Rotate X + Move Z, Rotate Y + Move Z]', 
                  fontsize=12, fontweight='bold')
    ax1.legend(loc='upper left', fontsize=9)
    
    # Set equal aspect ratio
    max_range = 1.5
    ax1.set_xlim([-0.5, max_range])
    ax1.set_ylim([-0.5, max_range])
    ax1.set_zlim([-0.5, max_range])
    
    # 2D projection showing XY plane
    ax2 = fig.add_subplot(122)
    ax2.scatter(0, 0, color='green', s=200, marker='o', label='Origin', zorder=5)
    ax2.scatter(pos_AB[0], pos_AB[1], color='blue', s=200, marker='s', label='A→B', zorder=5)
    ax2.scatter(pos_BA[0], pos_BA[1], color='red', s=200, marker='^', label='B→A', zorder=5)
    
    ax2.arrow(pos_AB[0], pos_AB[1], pos_BA[0]-pos_AB[0], pos_BA[1]-pos_AB[1],
              head_width=0.05, head_length=0.05, fc='purple', ec='purple',
              linewidth=2, label='Commutator')
    
    ax2.set_xlabel('X', fontsize=11)
    ax2.set_ylabel('Y', fontsize=11)
    ax2.set_title('XY Plane Projection\n(Top View)', fontsize=12, fontweight='bold')
    ax2.grid(True, alpha=0.3)
    ax2.axis('equal')
    ax2.legend(fontsize=10)
    
    plt.tight_layout()
    plt.show()
    
    # Print numerical results
    print("\nSE(3) Non-Commutativity Results:")
    print("\nPath A→B (Rotate X, then Rotate Y):")
    print(f"  Final position: [{pos_AB[0]:.3f}, {pos_AB[1]:.3f}, {pos_AB[2]:.3f}]")
    print("\nPath B→A (Rotate Y, then Rotate X):")
    print(f"  Final position: [{pos_BA[0]:.3f}, {pos_BA[1]:.3f}, {pos_BA[2]:.3f}]")
    print("\nCommutator [A, B]:")
    print(f"  Displacement: [{pos_BA[0]-pos_AB[0]:.3f}, {pos_BA[1]-pos_AB[1]:.3f}, {pos_BA[2]-pos_AB[2]:.3f}]")
    print(f"  Distance: {np.linalg.norm(pos_BA[:3] - pos_AB[:3]):.3f}")

# Run the simulation
simulate_se3_arm()

### Key Insight: Claws (Robotics)

The two paths lead to **completely different end-effector positions**, even though they use the same operations! This is why:
- Robot programming requires precise sequence specification
- Forward kinematics is path-dependent
- Inverse kinematics can have multiple solutions

In industrial robotics, the order of joint movements critically affects:
- Collision avoidance
- Energy efficiency
- Task completion success

---

# 4. Kets: Quantum Unitary Groups ($SU(2)$)

## The Problem

Quantum computing relies on manipulating qubits using **quantum gates**. These gates are unitary transformations on the Bloch sphere.

## The Mathematics

The Pauli matrices generate rotations in $SU(2)$:

$$\sigma_x = \begin{bmatrix} 0 & 1 \\ 1 & 0 \end{bmatrix}, \quad
\sigma_y = \begin{bmatrix} 0 & -i \\ i & 0 \end{bmatrix}, \quad
\sigma_z = \begin{bmatrix} 1 & 0 \\ 0 & -1 \end{bmatrix}$$

These matrices **do not commute**:

$$[\sigma_x, \sigma_y] = 2i\sigma_z$$

Applying an X-gate followed by a Y-gate produces a different quantum state than Y followed by X. The difference is a rotation around the Z-axis.

This non-commutativity is fundamental to:
- Quantum error correction
- Quantum algorithms (e.g., Grover, Shor)
- The Heisenberg uncertainty principle

## The Code (using PennyLane)

In [ ]:
# Check if PennyLane is available
try:
    import pennylane as qml
    PENNYLANE_AVAILABLE = True
except ImportError:
    PENNYLANE_AVAILABLE = False
    print("PennyLane not available. Install with: pip install pennylane")
    print("Showing manual implementation instead...\n")

if PENNYLANE_AVAILABLE:
    # PennyLane implementation
    dev = qml.device("default.qubit", wires=1)
    
    @qml.qnode(dev)
    def order_XY(theta):
        """Apply X-gate then Y-gate."""
        qml.RX(theta, wires=0)
        qml.RY(theta, wires=0)
        return [qml.expval(qml.PauliX(0)), 
                qml.expval(qml.PauliY(0)), 
                qml.expval(qml.PauliZ(0))]
    
    @qml.qnode(dev)
    def order_YX(theta):
        """Apply Y-gate then X-gate."""
        qml.RY(theta, wires=0)
        qml.RX(theta, wires=0)
        return [qml.expval(qml.PauliX(0)), 
                qml.expval(qml.PauliY(0)), 
                qml.expval(qml.PauliZ(0))]
    
    def visualize_quantum_commutator():
        """Visualize quantum gate non-commutativity on the Bloch sphere."""
        theta = np.pi / 2  # 90-degree rotations
        
        # Get final Bloch vectors for each ordering
        pos_XY = np.array(order_XY(theta))
        pos_YX = np.array(order_YX(theta))
        
        # Also compute intermediate states for visualization
        @qml.qnode(dev)
        def after_X(theta):
            qml.RX(theta, wires=0)
            return [qml.expval(qml.PauliX(0)), qml.expval(qml.PauliY(0)), qml.expval(qml.PauliZ(0))]
        
        @qml.qnode(dev)
        def after_Y(theta):
            qml.RY(theta, wires=0)
            return [qml.expval(qml.PauliX(0)), qml.expval(qml.PauliY(0)), qml.expval(qml.PauliZ(0))]
        
        pos_X = np.array(after_X(theta))
        pos_Y = np.array(after_Y(theta))
        
        # Create Bloch sphere visualization
        fig = plt.figure(figsize=(16, 7))
        
        # Left plot: 3D Bloch sphere
        ax1 = fig.add_subplot(121, projection='3d')
        
        # Draw Bloch sphere wireframe
        u, v = np.mgrid[0:2*np.pi:30j, 0:np.pi:20j]
        x_sphere = np.cos(u) * np.sin(v)
        y_sphere = np.sin(u) * np.sin(v)
        z_sphere = np.cos(v)
        ax1.plot_wireframe(x_sphere, y_sphere, z_sphere, color="gray", alpha=0.1, linewidth=0.5)
        
        # Draw coordinate axes
        ax1.quiver(0, 0, 0, 1.2, 0, 0, color='red', alpha=0.3, arrow_length_ratio=0.1)
        ax1.quiver(0, 0, 0, 0, 1.2, 0, color='green', alpha=0.3, arrow_length_ratio=0.1)
        ax1.quiver(0, 0, 0, 0, 0, 1.2, color='blue', alpha=0.3, arrow_length_ratio=0.1)
        ax1.text(1.3, 0, 0, '$|+\\rangle$', fontsize=12)
        ax1.text(0, 1.3, 0, '$|+i\\rangle$', fontsize=12)
        ax1.text(0, 0, 1.3, '$|0\\rangle$', fontsize=12)
        
        # Initial state |0>
        ax1.scatter(0, 0, 1, color='green', s=200, marker='o', label='|0⟩ (Initial)', zorder=5)
        
        # Path X→Y
        ax1.quiver(0, 0, 1, pos_X[0], pos_X[1], pos_X[2]-1, 
                   color='blue', alpha=0.4, arrow_length_ratio=0.1)
        ax1.quiver(pos_X[0], pos_X[1], pos_X[2], 
                   pos_XY[0]-pos_X[0], pos_XY[1]-pos_X[1], pos_XY[2]-pos_X[2],
                   color='blue', arrow_length_ratio=0.1, linewidth=2, label='X→Y')
        
        # Path Y→X
        ax1.quiver(0, 0, 1, pos_Y[0], pos_Y[1], pos_Y[2]-1, 
                   color='red', alpha=0.4, arrow_length_ratio=0.1)
        ax1.quiver(pos_Y[0], pos_Y[1], pos_Y[2], 
                   pos_YX[0]-pos_Y[0], pos_YX[1]-pos_Y[1], pos_YX[2]-pos_Y[2],
                   color='red', arrow_length_ratio=0.1, linewidth=2, label='Y→X')
        
        # Commutator difference
        ax1.plot([pos_XY[0], pos_YX[0]], [pos_XY[1], pos_YX[1]], [pos_XY[2], pos_YX[2]], 
                 'purple', linestyle='--', linewidth=3, marker='o', markersize=10,
                 label='Commutator')
        
        ax1.set_xlabel('X (σₓ)', fontsize=11)
        ax1.set_ylabel('Y (σᵧ)', fontsize=11)
        ax1.set_zlabel('Z (σᵤ)', fontsize=11)
        ax1.set_title('Quantum Gates on the Bloch Sphere\n[Rₓ(π/2), Rᵧ(π/2)] ≠ 0', 
                      fontsize=12, fontweight='bold')
        ax1.legend(loc='upper left', fontsize=10)
        ax1.set_xlim([-1.2, 1.2])
        ax1.set_ylim([-1.2, 1.2])
        ax1.set_zlim([-1.2, 1.2])
        
        # Right plot: XY plane projection
        ax2 = fig.add_subplot(122)
        
        # Draw unit circle
        circle = plt.Circle((0, 0), 1, fill=False, color='gray', linestyle='--', alpha=0.3)
        ax2.add_patch(circle)
        
        # Plot states
        ax2.scatter(0, 0, color='green', s=200, marker='o', label='|0⟩', zorder=5)
        ax2.scatter(pos_XY[0], pos_XY[1], color='blue', s=200, marker='s', label='X→Y', zorder=5)
        ax2.scatter(pos_YX[0], pos_YX[1], color='red', s=200, marker='^', label='Y→X', zorder=5)
        
        # Draw arrows
        ax2.arrow(0, 0, pos_XY[0], pos_XY[1], head_width=0.05, head_length=0.05, 
                  fc='blue', ec='blue', alpha=0.5, linewidth=1.5)
        ax2.arrow(0, 0, pos_YX[0], pos_YX[1], head_width=0.05, head_length=0.05, 
                  fc='red', ec='red', alpha=0.5, linewidth=1.5)
        ax2.plot([pos_XY[0], pos_YX[0]], [pos_XY[1], pos_YX[1]], 
                 'purple', linestyle='--', linewidth=2, marker='o', markersize=8)
        
        ax2.set_xlabel('X expectation ⟨σₓ⟩', fontsize=11)
        ax2.set_ylabel('Y expectation ⟨σᵧ⟩', fontsize=11)
        ax2.set_title('XY Plane Projection', fontsize=12, fontweight='bold')
        ax2.grid(True, alpha=0.3)
        ax2.axis('equal')
        ax2.legend(fontsize=10)
        ax2.set_xlim([-1.2, 1.2])
        ax2.set_ylim([-1.2, 1.2])
        
        plt.tight_layout()
        plt.show()
        
        # Print results
        print("\nQuantum Gate Non-Commutativity (SU(2)):")
        print(f"\nAngle: θ = π/2 ({np.degrees(theta):.0f}°)")
        print("\nPath X→Y:")
        print(f"  Bloch vector: [{pos_XY[0]:.4f}, {pos_XY[1]:.4f}, {pos_XY[2]:.4f}]")
        print("\nPath Y→X:")
        print(f"  Bloch vector: [{pos_YX[0]:.4f}, {pos_YX[1]:.4f}, {pos_YX[2]:.4f}]")
        print("\nCommutator [X, Y]:")
        print(f"  Difference: [{pos_YX[0]-pos_XY[0]:.4f}, {pos_YX[1]-pos_XY[1]:.4f}, {pos_YX[2]-pos_XY[2]:.4f}]")
        print(f"  Angular separation: {np.degrees(np.arccos(np.dot(pos_XY, pos_YX))):.2f}°")
        print("\n  → Different gate orders produce different quantum states!")
        print("  → This is why quantum algorithm design is so subtle.")
    
    visualize_quantum_commutator()
    
else:
    # Manual implementation without PennyLane
    print("Showing manual implementation of quantum gates...\n")
    
    def rotation_matrix_x(theta):
        """Rotation around X-axis (Pauli X)."""
        return np.array([[np.cos(theta/2), -1j*np.sin(theta/2)],
                        [-1j*np.sin(theta/2), np.cos(theta/2)]])
    
    def rotation_matrix_y(theta):
        """Rotation around Y-axis (Pauli Y)."""
        return np.array([[np.cos(theta/2), -np.sin(theta/2)],
                        [np.sin(theta/2), np.cos(theta/2)]])
    
    def state_to_bloch(state):
        """Convert quantum state to Bloch vector."""
        # Pauli matrices
        sigma_x = np.array([[0, 1], [1, 0]])
        sigma_y = np.array([[0, -1j], [1j, 0]])
        sigma_z = np.array([[1, 0], [0, -1]])
        
        x = np.real(state.conj() @ sigma_x @ state)
        y = np.real(state.conj() @ sigma_y @ state)
        z = np.real(state.conj() @ sigma_z @ state)
        return np.array([x, y, z])
    
    # Initial state |0>
    psi_0 = np.array([1, 0])
    theta = np.pi / 2
    
    # Path 1: X then Y
    Rx = rotation_matrix_x(theta)
    Ry = rotation_matrix_y(theta)
    psi_XY = Ry @ (Rx @ psi_0)
    bloch_XY = state_to_bloch(psi_XY)
    
    # Path 2: Y then X
    psi_YX = Rx @ (Ry @ psi_0)
    bloch_YX = state_to_bloch(psi_YX)
    
    print(f"Rotation angle: θ = π/2 (90°)\n")
    print(f"Path X→Y Bloch vector: [{bloch_XY[0]:.4f}, {bloch_XY[1]:.4f}, {bloch_XY[2]:.4f}]")
    print(f"Path Y→X Bloch vector: [{bloch_YX[0]:.4f}, {bloch_YX[1]:.4f}, {bloch_YX[2]:.4f}]")
    print(f"\nDifference: {np.linalg.norm(bloch_XY - bloch_YX):.4f}")
    print("\n→ Gate order matters in quantum computing!")

### Key Insight: Kets (Quantum Computing)

The two gate sequences produce **different quantum states** on the Bloch sphere. This non-commutativity is not a bug—it's a fundamental feature of quantum mechanics!

This connects to:
- **Heisenberg Uncertainty Principle**: $[\hat{x}, \hat{p}] = i\hbar$
- **Quantum Error Correction**: Must account for gate ordering
- **Quantum Algorithms**: Gate sequence optimization is critical
- **Berry Phase**: Geometric phases from cyclic evolution

---

# 5. Continuous: Neural ODEs (Diffeomorphisms)

## The Problem

Modern deep learning uses discrete layers: $f_3 \circ f_2 \circ f_1(x)$. But what if we made depth **continuous**?

## The Mathematics

A **Neural ODE** treats the network as a continuous transformation:

$$\frac{dh(t)}{dt} = f(h(t), t)$$

Inference is integration: $h(T) = h(0) + \int_0^T f(h(t), t) dt$

When we have multiple vector fields $f_A$ and $f_B$, their composition generally does **not commute**:

$$\int f_A \circ \int f_B \neq \int f_B \circ \int f_A$$

The network topology matters! Different orderings of transformations lead to different outputs.

## The Code

In [ ]:
def vector_field_A(t, state):
    """
    Vector field A: Rotation
    Represents a neural network layer that rotates features.
    Matrix form: [[0, -1], [1, 0]]
    """
    x, y = state
    return np.array([-y, x])

def vector_field_B(t, state):
    """
    Vector field B: Shearing
    Represents a neural network layer that shears features.
    Matrix form: [[0, 1], [1, 0]]
    """
    x, y = state
    return np.array([y, x])

def run_neural_ode(duration=1.5, resolution=100):
    """
    Compare two different orderings of neural ODE flows.
    
    Parameters:
    -----------
    duration : float
        Integration time for each flow
    resolution : int
        Number of evaluation points
    """
    initial_state = np.array([1.0, 0.0])
    t_span = (0, duration)
    t_eval = np.linspace(0, duration, resolution)
    
    # Path 1: Flow through A, then flow through B
    sol_A = solve_ivp(vector_field_A, t_span, initial_state, t_eval=t_eval, method='RK45')
    mid_state_1 = sol_A.y[:, -1]
    sol_AB = solve_ivp(vector_field_B, t_span, mid_state_1, t_eval=t_eval, method='RK45')
    
    # Path 2: Flow through B, then flow through A
    sol_B = solve_ivp(vector_field_B, t_span, initial_state, t_eval=t_eval, method='RK45')
    mid_state_2 = sol_B.y[:, -1]
    sol_BA = solve_ivp(vector_field_A, t_span, mid_state_2, t_eval=t_eval, method='RK45')
    
    # Create visualization
    fig = plt.figure(figsize=(18, 5))
    
    # Left: Complete trajectories
    ax1 = fig.add_subplot(131)
    ax1.plot(sol_A.y[0], sol_A.y[1], 'b--', linewidth=2, alpha=0.5, label='Field A')
    ax1.plot(sol_AB.y[0], sol_AB.y[1], 'b-', linewidth=2, label='A→B complete')
    ax1.plot(sol_B.y[0], sol_B.y[1], 'r--', linewidth=2, alpha=0.5, label='Field B')
    ax1.plot(sol_BA.y[0], sol_BA.y[1], 'r-', linewidth=2, label='B→A complete')
    
    ax1.scatter([initial_state[0]], [initial_state[1]], c='green', s=200, 
                marker='o', label='Input', zorder=5)
    ax1.scatter([sol_AB.y[0, -1]], [sol_AB.y[1, -1]], c='blue', s=200, 
                marker='s', label='A→B output', zorder=5)
    ax1.scatter([sol_BA.y[0, -1]], [sol_BA.y[1, -1]], c='red', s=200, 
                marker='^', label='B→A output', zorder=5)
    
    ax1.set_xlabel('Feature 1', fontsize=11)
    ax1.set_ylabel('Feature 2', fontsize=11)
    ax1.set_title('Neural ODE Flow Trajectories', fontsize=12, fontweight='bold')
    ax1.grid(True, alpha=0.3)
    ax1.legend(fontsize=9, loc='best')
    ax1.axis('equal')
    
    # Middle: Final states with commutator
    ax2 = fig.add_subplot(132)
    ax2.scatter([initial_state[0]], [initial_state[1]], c='green', s=200, 
                marker='o', label='Input', zorder=5)
    ax2.scatter([sol_AB.y[0, -1]], [sol_AB.y[1, -1]], c='blue', s=200, 
                marker='s', label='A→B', zorder=5)
    ax2.scatter([sol_BA.y[0, -1]], [sol_BA.y[1, -1]], c='red', s=200, 
                marker='^', label='B→A', zorder=5)
    
    # Draw the commutator difference
    ax2.arrow(sol_AB.y[0, -1], sol_AB.y[1, -1], 
              sol_BA.y[0, -1] - sol_AB.y[0, -1], 
              sol_BA.y[1, -1] - sol_AB.y[1, -1],
              color='purple', width=0.05, head_width=0.2, head_length=0.2,
              label='Commutator', zorder=4)
    
    ax2.set_xlabel('Feature 1', fontsize=11)
    ax2.set_ylabel('Feature 2', fontsize=11)
    ax2.set_title('Non-Commutativity\nof Layer Ordering', fontsize=12, fontweight='bold')
    ax2.grid(True, alpha=0.3)
    ax2.legend(fontsize=10)
    ax2.axis('equal')
    
    # Right: Vector fields
    ax3 = fig.add_subplot(133)
    
    # Create grid for vector field visualization
    x_range = np.linspace(-2, 4, 15)
    y_range = np.linspace(-2, 4, 15)
    X_grid, Y_grid = np.meshgrid(x_range, y_range)
    
    # Compute vector field A
    U_A = -Y_grid
    V_A = X_grid
    
    # Compute vector field B
    U_B = Y_grid
    V_B = X_grid
    
    # Plot both vector fields
    ax3.quiver(X_grid, Y_grid, U_A, V_A, alpha=0.3, color='blue', 
               scale=30, width=0.003, label='Field A (Rotation)')
    ax3.quiver(X_grid, Y_grid, U_B, V_B, alpha=0.3, color='red', 
               scale=30, width=0.003, label='Field B (Shear)')
    
    ax3.scatter([initial_state[0]], [initial_state[1]], c='green', s=200, 
                marker='o', label='Input', zorder=5)
    
    ax3.set_xlabel('Feature 1', fontsize=11)
    ax3.set_ylabel('Feature 2', fontsize=11)
    ax3.set_title('Vector Fields\n(Neural Network Layers)', fontsize=12, fontweight='bold')
    ax3.grid(True, alpha=0.3)
    ax3.legend(fontsize=9)
    ax3.axis('equal')
    ax3.set_xlim([-2, 4])
    ax3.set_ylim([-2, 4])
    
    plt.tight_layout()
    plt.show()
    
    # Print results
    print("\nNeural ODE Non-Commutativity Results:")
    print(f"\nInitial state: [{initial_state[0]:.3f}, {initial_state[1]:.3f}]")
    print(f"Integration time per layer: {duration:.2f}")
    
    print("\nPath A→B (Rotate then Shear):")
    print(f"  After A: [{mid_state_1[0]:.3f}, {mid_state_1[1]:.3f}]")
    print(f"  Final:   [{sol_AB.y[0, -1]:.3f}, {sol_AB.y[1, -1]:.3f}]")
    
    print("\nPath B→A (Shear then Rotate):")
    print(f"  After B: [{mid_state_2[0]:.3f}, {mid_state_2[1]:.3f}]")
    print(f"  Final:   [{sol_BA.y[0, -1]:.3f}, {sol_BA.y[1, -1]:.3f}]")
    
    commutator_norm = np.linalg.norm([sol_BA.y[0, -1] - sol_AB.y[0, -1],
                                      sol_BA.y[1, -1] - sol_AB.y[1, -1]])
    print(f"\nCommutator magnitude: {commutator_norm:.3f}")
    print("\n  → Different layer orderings produce different outputs!")
    print("  → This is why neural architecture search is important.")
    print("  → Neural ODEs make this continuous rather than discrete.")

# Run the simulation
run_neural_ode(duration=1.5, resolution=100)

### Key Insight: Continuous Neural Networks

Even in continuous systems, **order matters**. Flowing through layer A then B produces a different result than B then A.

This connects to:
- **Neural Architecture Search**: Layer ordering is a design choice
- **Residual Connections**: Skip connections change the effective flow
- **Normalizing Flows**: Invertibility requires careful composition
- **Continuous Depth**: Neural ODEs make depth a continuous hyperparameter

---

# Conclusion: The Unity of Non-Commutativity

We've seen the same mathematical structure—**non-commutative algebra**—appear across five completely different domains:

1. **Control Theory**: Cars parking sideways
2. **Biomechanics**: Cats landing on their feet
3. **Robotics**: Robot arms with path-dependent positioning
4. **Quantum Mechanics**: Quantum gates creating geometric phases
5. **Machine Learning**: Neural ODEs with order-dependent outputs

## The Common Thread: Lie Theory

All these examples are instances of **Lie groups** and **Lie algebras**:

- $SE(2)$, $SE(3)$: **Special Euclidean Groups** (rigid body motion)
- $SO(3)$: **Special Orthogonal Group** (rotations)
- $SU(2)$: **Special Unitary Group** (quantum states)
- Diff: **Diffeomorphism Group** (smooth transformations)

The commutator $[A, B] = AB - BA$ is the fundamental operation in Lie algebras, and it measures how much operations fail to commute.

## Practical Implications

Understanding non-commutativity is essential for:

- **Engineering**: Control systems, robotics, aerospace
- **Physics**: Quantum mechanics, gauge theory, general relativity
- **Computer Science**: Graphics, optimization, machine learning
- **Mathematics**: Differential geometry, topology, representation theory

## Further Reading

- **Control Theory**: Brockett's "Asymptotic Stability and Feedback Stabilization"
- **Gauge Theory**: Montgomery, "A Tour of Subriemannian Geometries"
- **Robotics**: Murray, Li, Sastry, "A Mathematical Introduction to Robotic Manipulation"
- **Quantum Computing**: Nielsen & Chuang, "Quantum Computation and Quantum Information"
- **Neural ODEs**: Chen et al., "Neural Ordinary Differential Equations" (NeurIPS 2018)

---

**The Central Dogma**: The order of operations matters. This is not a bug—it's a fundamental feature of reality.

$$[A, B] \neq 0 \implies \text{Order Creates New Dimensions}$$